In [1]:
import json, requests
import pandas as pd
from tqdm import tqdm
import time
import numpy as np
import geopandas as geopd
import warnings
warnings.filterwarnings('ignore')

## Business Understanding

### Problem Framing

## Getting data

In [83]:
def get_geo_data():
        
    print('> Creating the geo_dataframe.')
    
    geo_df = geopd.read_file('https://data.cityofnewyork.us/api/geospatial/d3qk-pfyz?method=export&format=GeoJSON')
    geo_df['centroid_gpd'] = geo_df.geometry.centroid
    geo_df['lat_long'] = 0
    
    print('> Getting the centroids coordinates (lat/long).')
    
    time.sleep(0.7)
    
    for i in tqdm(range(len(geo_df)),ncols=150):
        
        centroid_lat_long = np.array(geo_df.centroid_gpd[i])
        geo_df.loc[i,'lat_long'] = [str(centroid_lat_long[1])+','+str(centroid_lat_long[0])]
    
    print('> Done.')
    
    return geo_df

def get_foursquare_search_data(location):

    params = dict(
        client_id='UTCC2TLL2HALAQM2CAP3UK4SPVA4PSJFP32OW3WKD0PJAFLF',
        client_secret='TMRMFTUYEWR3E5ISBAGFIXX10FZE0TVVSPL5LPRLUWSQZEAW',
        ll = location,
        radius = 5000,
        limit = 100,
        v = '20191122',
        intent='browse',
        categoryId = '4bf58dd8d48988d1ca941735')
    
    url = 'https://api.foursquare.com/v2/venues/search?'
    pizzeria_data = requests.get(url = url, params= params)
    
    if pizzeria_data.ok:
        
        #return pizzeria_data.json()['response']['venues']
        return pd.io.json.json_normalize(pizzeria_data.json()['response']['venues'])
    
    else:
        
        raise Exception('Could not get the data: ',pizzeria_data.status_code)
        
        
def get_pizzerias_from_nta_centroids(centroids):
    
    pizza_nta = []
        
    for centroid in tqdm(centroids):
        
        data = pizza_nta.append(get_foursquare_search_data(location=centroid))
    
    df = pd.DataFrame()
        
    for i in tqdm(range(len(pizza_nta))):
        df = pd.concat([df,pd.DataFrame.from_dict(pizza_nta[i])])
        
    df.reset_index(drop=True,inplace=True)    
    
    return df

In [84]:
geo_df2 = get_geo_data()

> Creating the geo_dataframe.
> Getting the centroids coordinates (lat/long).


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 195/195 [00:00<00:00, 2429.96it/s]

> Done.


In [88]:
df = get_pizzerias_from_nta_centroids(geo_df2.lat_long)

100%|██████████| 195/195 [00:01<00:00, 147.62it/s]


## Data Preparation

## Modeling

## Evaluation

## Results